In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
import tensorflow as tf

from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.constraints import max_norm
from keras.applications.imagenet_utils import decode_predictions
from PIL import Image
# from efficientnet.keras import EfficientNetB7
# from efficientnet.keras import center_crop_and_resize, preprocess_input
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [ ]:
# tsvファイルの読み込み
train_master = pd.read_csv('train_master.tsv', sep='\t')
label_master = pd.read_csv('label_master.tsv', sep='\t')

master = pd.merge(train_master, label_master, on='label_id', how='inner').sort_values('label_id')
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")


## データの拡張

ImageDataGeneratorを用いてデータを拡張

### 引数

- featurewise_center: bool．
    - データセット全体で，入力の平均を0にします．
- samplewise_center: 
    - 真理値．各サンプルの平均を0にします．
- featurewise_std_normalization: bool.
    - 入力をデータセットの標準偏差で正規化します．
- samplewise_std_normalization: bool．
    - 各入力をその標準偏差で正規化します．
- zca_epsilon: ZCA白色化のイプシロン．
    - デフォルトは1e-6．
- zca_whitening: bool．
    - ZCA白色化を適用します．
- rotation_range: int．
    - 画像をランダムに回転する回転範囲．
- width_shift_range: float（横幅に対する割合）．
    - ランダムに水平シフトする範囲．
- height_shift_range: float（縦幅に対する割合）．
    - ランダムに垂直シフトする範囲．
- shear_range: float．
    - シアー強度（反時計回りのシアー角度）．
- zoom_range: float or [lower，upper]．
    - ランダムにズームする範囲．
    - floatが与えられた場合，
        - [lower, upper] = [1-zoom_range, 1+zoom_range]です．
- channel_shift_range: float．
    - ランダムにチャンネルをシフトする範囲．
- fill_mode: {"constant", "nearest", "reflect", "wrap"}のいずれか．
    - デフォルトは 'nearest'です．
    - 指定されたモードに応じて，入力画像の境界周りを埋めます．
        - "constant": kkkkkkkk|abcd|kkkkkkkk (cval=k)
        - "nearest": aaaaaaaa|abcd|dddddddd
        - "reflect": abcddcba|abcd|dcbaabcd
        - "wrap": abcdabcd|abcd|abcdabcd
- cval: float or int．
    - fill_mode = "constant"のときに境界周辺で利用される値．
- horizontal_flip: bool．
    - 水平方向に入力をランダムに反転します．
- vertical_flip: bool．
    - 垂直方向に入力をランダムに反転します．
- rescale: 画素値のリスケーリング係数．
    - デフォルトはNone．
    - Noneか0ならば，適用しない．
    - それ以外であれば，(他の変換を行う前に) 与えられた値をデータに積算する．
- preprocessing_function: 各入力に適用される関数です．
    - この関数は他の変更が行われる前に実行されます．
    - この関数は3次元のNumpyテンソルを引数にとり，同じshapeのテンソルを出力するように定義する必要があります．
- data_format: {"channels_first", "channels_last"}のどちらか．
    - "channels_last"の場合
        - 入力のshapeは(samples, height, width, channels)となります.   
    - "channels_first"の場合
        - (samples, channels, height, width)となります．
    - デフォルトはKerasの設定ファイル~/.keras/keras.jsonのimage_data_formatの値です．
    - 一度も値を変更していなければ，"channels_last"になります．
validation_split: float．
    - 検証のために予約しておく画像の割合（厳密には0から1の間）です．

- 参考サイト
    - https://keras.io/ja/preprocessing/image/

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
y_train = np_utils.to_categorical(y_train, num_classes)


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=epochs)